# Chapter 3 Function
## 19. 不要把函数返回的多个数值返回到三个以上的变量

In [1]:
# 有多个返回值时，用带‘*’的表达式接受那些没有被普通变量捕获的值（元组类型）
def get_avg_retion(numbers: list):
    average = sum(numbers)/len(numbers)
    scaled = [x / average for x in numbers]
    scaled.sort(reverse=True)
    return scaled
length = [63, 73, 72, 60, 67, 66, 71, 62, 72, 70]
longest, *middle, shortest = get_avg_retion(length)
print(f'longest : {longest},\n,middle:{middle}, \nshortest:{shortest}')

longest : 1.0798816568047338,
,middle:[1.0650887573964498, 1.0650887573964498, 1.0502958579881658, 1.0355029585798818, 0.9911242603550297, 0.9763313609467457, 0.9319526627218936, 0.9171597633136096], 
shortest:0.8875739644970415


## 20. 遇到意外情况时应该抛出异常，不要返回None
返回None会和0值冲突

In [2]:
def careful_divide(a:float,b: float) -> float:
    """Divides a by b

    Raises:
    ValueError: when inputs cannot be divide
    """
    try:
        return a / b
    except ZeroDivisionError:
        raise ValueError('Invalid inputs')

In [3]:
try:
    result = careful_divide(5,2)
except ValueError:
    print('非法输出')
else:
    print(f'result is {result}')

result is 2.5


## 21. 了解如何在闭包里使用外围作用域的变量
避免函数中的局部变量污染外部模块，函数内作用域若有该变量则直接赋值，若无则以此函数范围作作用域创建变量

In [8]:
def sort_priority(values:list, group: list):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [9]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


found的作用域在闭包函数内，赋值在外部无效，若想实现需要使用nonlocal声明

In [10]:
def sort_priority(values:list, group: list):
    found = False
    def helper(x):
        if x in group:
            return (0, x)
        found = True
        return (1, x)
    values.sort(key=helper)
    return found

In [12]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
f = sort_priority(numbers, group)
print(numbers,f)

[2, 3, 5, 7, 1, 4, 6, 8] False


In [13]:
def sort_priority(values:list, group: list):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            return (0, x)
        found = True
        return (1, x)
    values.sort(key=helper)
    return found
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
f = sort_priority(numbers, group)
print(numbers,f)

[2, 3, 5, 7, 1, 4, 6, 8] True


还可以使用__call__方法定义类的调用触发函数，免去闭包

In [16]:
class Sorted():
    def __init__(self, group):
        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

In [17]:
sortd = Sorted(group)
numbers.sort(key=sortd)
print(sortd.found)

True


## 22. 用数量可变的位置参数给函数设计清晰的参数列表

In [1]:
def log(message, values):
    if not  values:
        print(message)
    else:
        values_str = ','.join(str(x) for x in values)
        print(f'{message} : {values_str}')

In [2]:
log('My numbers are', [1, 2])
log('Hi there', [])

My numbers are : 1,2
Hi there


此时第二个参数如果不传会报错

In [3]:
log('Hi there')

TypeError: log() missing 1 required positional argument: 'values'

使用'*’在函数的位置参数中，这样只需要提供不带*参数，并不会报错

In [4]:
def log1(message, *values):  # 将所有输入的数值pack到values中
    if not  values:
        print(message)
    else:
        values_str = ','.join(str(x) for x in values)
        print(f'{message} : {values_str}')

In [9]:
log1('My numbers are', 1, 2)

My numbers are : 1,2


In [15]:
log1('My numbers are', [1, 2])
# 此时pack了一个list，及[[1, 2]],如果想给list进行传入，则需要继续使用‘*’
log1('My numbers are', *[1, 2])

My numbers are : [1, 2]
My numbers are : 1,2


In [12]:
log1('Hi there')

Hi there
